In [1]:
import matplotlib.pyplot as plt
import os
import shutil

In [2]:
classes = os.listdir("Data")

In [3]:
classes

['Aerodactyl',
 'Bulbasaur',
 'Charmander',
 'Dratini',
 'Fearow',
 'Meowth',
 'Pikachu',
 'Psyduck',
 'Spearow',
 'Squirtle']

In [4]:
os.mkdir('val')

In [5]:
for i in classes:
    images = os.listdir(os.path.join("./Data", i))
    length = int(.1*len(images))
    os.mkdir("val/"+i)
    for j in images[:length]:
        shutil.move("./Data/"+i+"/"+j, "./val/"+i)

In [6]:
for i in classes:
    images1 = os.listdir(os.path.join("./Data", i))
    print("train", i, len(images1))
    images2 = os.listdir(os.path.join('./val', i))
    print("val", i, len(images2))

train Aerodactyl 72
val Aerodactyl 8
train Bulbasaur 230
val Bulbasaur 25
train Charmander 238
val Charmander 26
train Dratini 89
val Dratini 9
train Fearow 100
val Fearow 11
train Meowth 57
val Meowth 6
train Pikachu 242
val Pikachu 26
train Psyduck 129
val Psyduck 14
train Spearow 106
val Spearow 11
train Squirtle 224
val Squirtle 24


Image Data Generator

In [7]:
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input

In [9]:
model = MobileNet(weights="imagenet", input_shape=(224, 224, 3), include_top=False)

17235968/17225924 [==============================] - 19s 1us/step


In [10]:
model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32

In [11]:
from keras.preprocessing.image import ImageDataGenerator

In [12]:
train = ImageDataGenerator(preprocessing_function=preprocess_input)
val = ImageDataGenerator(preprocessing_function=preprocess_input)

In [13]:
tr_gen = train.flow_from_directory("Data/", target_size=(224, 224), batch_size=32, class_mode="categorical")

Found 1482 images belonging to 10 classes.


In [14]:
val_gen = val.flow_from_directory("val/", target_size=(224, 224), class_mode="categorical")

Found 160 images belonging to 10 classes.


Now we have a model whose dense layer has been removed. Now add layer using function api

In [15]:
from keras.models import Model
from keras.layers import *

In [16]:
model.output

<KerasTensor: shape=(None, 7, 7, 1024) dtype=float32 (created by layer 'conv_pw_13_relu')>

In [17]:
avl = GlobalAveragePooling2D()(model.output)
fc1 = Dense(256, activation='relu')(avl)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(10, activation="softmax")(d1)

In [18]:
complete_model = Model(model.input, fc2)

In [19]:
complete_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0     

In [20]:
for ix in range(88):
    complete_model.layers[ix].trainable = False

In [21]:
complete_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
history = complete_model.fit_generator(tr_gen, epochs=5, steps_per_epoch=50, validation_data=val_gen, validation_steps=5)

C:\Users\DELL\anaconda3\lib\site-packages\keras\engine\training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
41/50 [=======================>......] - ETA: 20s - loss: 3.1823 - accuracy: 0.1195

C:\Users\DELL\anaconda3\lib\site-packages\PIL\Image.py:951: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


50/50 [==============================] - 186s 3s/step - loss: 3.1282 - accuracy: 0.1253 - val_loss: 2.1026 - val_accuracy: 0.2375


In [23]:
import pandas as pd
import numpy as np
from keras.preprocessing import image

In [24]:
data = pd.read_csv("D:\VS\Coding-Blocks-ML\Pokedex\sample_submission.csv")

In [26]:
for idx, i in enumerate(data["Name"]):
    img_path = './Test/images/'+i
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    t = np.argmax(complete_model.predict(x))
    data.iloc[idx, 1] = t

In [27]:
data.to_csv('main2.csv', index=False)

In [28]:
val_gen.class_indices, tr_gen.class_indices

({'Aerodactyl': 0,
  'Bulbasaur': 1,
  'Charmander': 2,
  'Dratini': 3,
  'Fearow': 4,
  'Meowth': 5,
  'Pikachu': 6,
  'Psyduck': 7,
  'Spearow': 8,
  'Squirtle': 9},
 {'Aerodactyl': 0,
  'Bulbasaur': 1,
  'Charmander': 2,
  'Dratini': 3,
  'Fearow': 4,
  'Meowth': 5,
  'Pikachu': 6,
  'Psyduck': 7,
  'Spearow': 8,
  'Squirtle': 9})